In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import string,nltk
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

In [2]:
train=pd.read_csv("datasets/fake_news/train.csv")
test=pd.read_csv("datasets/fake_news/test.csv")


print(train.shape,test.shape)

(20800, 5) (5200, 4)


# Data Analysis

In [3]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [5]:
# It shows the distribution of features and labels.
train.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [6]:
# This function shows the information about the data:
# Whether some Columns have missing data 
# Data type of different columns
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


# Imputation

In [8]:

# How much data is missing ?
train.isnull().sum()
# Show count of 'NaN' in each column

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
test.isnull().sum()


id          0
title     122
author    503
text        7
dtype: int64

In [3]:
train1=train.dropna(axis=0,subset=['text'])
train1.shape

(20761, 5)

In [12]:
train1.isnull().sum()

id           0
title      558
author    1918
text         0
label        0
dtype: int64

In [5]:
train=train.fillna(' ')
test=test.fillna(' ')
train1=train1.fillna(' ')
train2=train1.copy()
train2


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
train['check']=train['author']+train['title']+train['text']
test['check']=test['author']+test['title']+test['text']
train1['check']=train1['author']+train1['title']+train1['text']
train2['check']=train2['author']+' '+train2['title']+' '+train2['text']
train2.head()


,id,title,author,text,label,check
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [7]:
train.drop('author',axis=1,inplace=True)
train.drop('title',axis=1,inplace=True)
train.drop('text',axis=1,inplace=True)


train1.drop('author',axis=1,inplace=True)
train1.drop('title',axis=1,inplace=True)
train1.drop('text',axis=1,inplace=True)

train2.drop('author',axis=1,inplace=True)
train2.drop('title',axis=1,inplace=True)
train2.drop('text',axis=1,inplace=True)


In [8]:
stop=stopwords.words('english')
stop=stop+list(string.punctuation)
lemm=WordNetLemmatizer()
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize,word_tokenize

In [9]:
for index, row in train.iterrows():
    clean=''
    sentence=row['check']
    words=word_tokenize(sentence)
    words=[w for w in words if not w in stop]
    
    for w in words:
        w=str(lemm.lemmatize(w))
        clean=clean+' ' +w.lower()
    train.loc[index,'check']=clean

    


In [10]:
for index1, row1 in train1.iterrows():
    clean1=''
    sentence1=row1['check']
    words1=word_tokenize(sentence1)
    words1=[w for w in words1 if not w in stop]
    
    for w in words1:
        w=str(lemm.lemmatize(w))
        clean1=clean1+' ' +w.lower()
    train1.loc[index1,'check']=clean1

In [11]:
for index2, row2 in train2.iterrows():
    clean2=''
    sentence2=row2['check']
    words2=word_tokenize(sentence2)
    words2=[w for w in words1 if not w in stop]
    
    for w in words2:
        w=str(lemm.lemmatize(w))
        clean2=clean2+' ' +w.lower()
    train2.loc[index2,'check']=clean2

In [12]:


from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=1000)
x = vectorizer.fit_transform(train['check'])
#print(vectorizer.get_feature_names())

vectorizer1 = TfidfVectorizer(ngram_range=(1,2),max_features=1000)
x1 = vectorizer1.fit_transform(train1['check'])

vectorizer2 = TfidfVectorizer(ngram_range=(1,2),max_features=1000)
x2 = vectorizer2.fit_transform(train2['check'])

In [13]:
y=train['label']
y1=train1['label']
y2=train2['label']
#test_data=vectorizer.transform(test['check'])
x_train,x_test,y_train,y_test=train_test_split(x,y)

x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1)

x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y2)
#x_train.shape

In [14]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(x_train,y_train)
log1=LogisticRegression()
log1.fit(x1_train,y1_train)
log2=LogisticRegression()
log2.fit(x2_train,y2_train)


print('Logistic regression on training data by replacing  null value')
print(log.score(x_train,y_train))
print('Logistic regression on training data by deleting  null entries')
print(log1.score(x1_train,y1_train))
print('Logistic regression on testing data by replacing null value')
print(log.score(x_test,y_test))
print('Logistic regression on testing data by deleting null entries')
print(log1.score(x1_test,y1_test))
print('Logistic regression on training data by deleting  null entries and adding with space')
print(log2.score(x2_train,y2_train))
print('Logistic regression on testing data by deleting null entries and adding with space')
print(log2.score(x2_test,y2_test))
#ylog=log.predict(vectorizer.fit_transform(test['check']))


Logistic regression on training data by replacing  null value
0.9564102564102565
Logistic regression on training data by deleting  null entries
0.9570327552986513
Logistic regression on testing data by replacing null value
0.9467307692307693
Logistic regression on testing data by deleting null entries
0.9466384126372568
Logistic regression on training data by deleting  null entries and adding with space
0.5000642260757868
Logistic regression on testing data by deleting null entries and adding with space
0.5010595261028703


In [31]:
#np.savetxt("res.csv",ylog,fmt="%.5f",delimiter=",")

In [16]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train,y_train)
svc1=SVC()
svc1.fit(x1_train,y1_train)
svc2=SVC()
svc2.fit(x2_train,y2_train)
print('SVM on training data by replacing null values')
print(svc.score(x_train,y_train))
print('SVM on training data by deleting null entries')
print(svc1.score(x1_train,y1_train))
print('SVM on testing data by replacing null values')
print(svc.score(x_test,y_test))
print('SVM on testing data by deleting null entries')
print(svc1.score(x1_test,y1_test))
print('SVM on training data by deleting null entries and adding with space')
print(svc2.score(x2_train,y2_train))
print('SVM on testing data by deleting null values and adding with space')
print(svc2.score(x2_test,y2_test))


SVM on training data by replacing null values
0.9939743589743589
SVM on training data by deleting null entries
0.9944765574823379
SVM on testing data by replacing null values
0.9567307692307693
SVM on testing data by deleting null entries
0.9601232903101522
SVM on training data by deleting null entries and adding with space
0.5000642260757868
SVM on testing data by deleting null values and adding with space
0.5010595261028703


In [17]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
rf1=RandomForestClassifier()
rf1.fit(x1_train,y1_train)
rf2=RandomForestClassifier()
rf2.fit(x2_train,y2_train)
print('Random Forest on training data by replacing null values')
print(rf.score(x_train,y_train))
print('Random Forest on training data by deleting null entries')
print(rf1.score(x1_train,y1_train))
print('Random Forest on testing data by replacing null values')
print(rf.score(x_test,y_test))
print('Random Forest on testing data by deleting null entries')
print(rf1.score(x1_test,y1_test))
print('Random Forest on training data by deleting null entries and concatenating with space')
print(rf2.score(x2_train,y2_train))
print('Random Forest on testing data by deleting null values and concatenating with space')
print(rf2.score(x2_test,y2_test))


Random Forest on training data by replacing null values
1.0
Random Forest on training data by deleting null entries
1.0
Random Forest on testing data by replacing null values
0.9567307692307693
Random Forest on testing data by deleting null entries
0.9560778270082836
Random Forest on training data by deleting null entries and concatenating with space
0.49993577392421323
Random Forest on testing data by deleting null values and concatenating with space
0.49894047389712964
